In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torch.utils.data as tdata
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

np.random.seed(0)
torch.manual_seed(0);

# Data Preprocessing

In [2]:
train_df = pd.read_csv('train.csv')
train_df.head(5)

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,1,6,14,6,6,1,1,10,2,1
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,12,2,2,4,1,3,1,11,15,1


In [3]:
# Get train and test data as np arrays
X0, Y = train_df.values[:, :-1], train_df.values[:, -1]
X0.shape, Y.shape

((592380, 27), (592380,))

In [4]:
# Only columns 3 and 4 (opened_position_qty and closed_position_qty) have NaN's
# Have to decide how to handle NaN's at some point
print(f'Cols where train data is Nan: {np.where(np.any(np.isnan(X0), axis=0))[0]}')

Cols where train data is Nan: [3 4]


### Handle Nan's by just ignoring those columns, lmao

In [5]:
# Handle Nans by dropping bad columns
def process(X0):
    '''
    Function that takes training / test data, 
    and process it for training / evaluation
    '''
    X = X0[:, 1:] # Don't need id lol
    X = X[:, np.all(~np.isnan(X), axis=0)]
    return X

X = process(X0)
(N, D) = X.shape

# Model attempt 1

In [6]:
### Just gonna take a random 10th for validation
val_size = N // 10
inds = np.random.permutation(N)
X, valX = X[inds[:-val_size]], X[inds[-val_size:]]
Y, valY = Y[inds[:-val_size]], Y[inds[-val_size:]]

In [7]:
batch_size = 32
X, Y = torch.Tensor(X), torch.Tensor(Y)
valX, valY = torch.Tensor(valX), torch.Tensor(valY)


train_loader = tdata.DataLoader(tdata.TensorDataset(X, Y), \
                                     batch_size=batch_size,\
                                     shuffle=True)
val_loader = tdata.DataLoader(tdata.TensorDataset(valX, valY), \
                                     batch_size=batch_size,\
                                     shuffle=True)

### Notes on sigmoid
Either Option 1:

1) No nn.Sigmoid() layer, 2) Use nn.BCEWithLogitsLoss, and 3) apply torch.sigmoid() to output to
get probabilities.
This option is more numerically stable

OR Option 2:

1) Use nn.Sigmoid() layer, 2) Use nn.BCELoss(), and 3) Don't need torch.sigmoid() at the prediction step

In [8]:
# Simple one hidden layer (k units) logistic regression
k = 128
model = nn.Sequential(
        nn.Linear(X.shape[1], k),
        nn.ReLU(), 
        nn.Dropout(0.3),
        nn.Linear(k, 1),
        #nn.Sigmoid(); #Option 2
    )

criterion = nn.BCEWithLogitsLoss()
#criterion = nn.BCELoss(); #Option 2
optimizer = optim.Adam(model.parameters())

In [9]:
# Some layers, such as Dropout, behave differently during training
model.train()

for epoch in range(3):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate loss (broadcast target to (B, 1), where B is batch size)
        loss = criterion(output, target.unsqueeze(1)) 

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

Train Epoch: 1  Loss: 0.6288
Train Epoch: 2  Loss: 0.6824
Train Epoch: 3  Loss: 0.6824


In [10]:
# Putting layers like Dropout into evaluation mode
model.eval()

val_loss = 0
correct = 0

# Turning off automatic differentiation
with torch.no_grad():
    for i, (data, target) in enumerate(val_loader):
        output = torch.sigmoid(model(data))
        # output = model(data); #Option 2
        
        val_loss += criterion(output, target.unsqueeze(1)).item()  # Sum up batch loss
        pred = output.round()
        correct += (pred.eq(target.view_as(pred))).sum().item()

val_loss /= len(val_loader.dataset)

print('Val set: Average loss: %.4f, Accuracy: %d/%d (%.4f)' %
      (val_loss, correct, len(val_loader.dataset),
       100. * correct / len(val_loader.dataset)))

Val set: Average loss: 0.0237, Accuracy: 37946/59238 (64.0569)


# Get Predictions on Test Set

In [11]:
test_df = pd.read_csv('test.csv')
tX = process(test_df.values)

with torch.no_grad():
    output = torch.sigmoid(model(torch.Tensor(tX)))
    #output = model(torch.Tensor(tX)); #Option 2
    
output_df = pd.DataFrame({'id':test_df['id'], 'Predicted':output.data.numpy().squeeze()})
output_df.to_csv('submission.csv', index=False)

# Old Code

In [12]:
# Get some data to make sure logistic regression worked lmao
import sys
sys.path.append('~/git/MLModels')
from MLModels import utils as u
from MLModels import linearModels as lm

f, line = u.genF(zero_one=True)
X, Y = u.genData(f, 10000)
(N, D) = X.shape